<a href="https://colab.research.google.com/github/tristan-quant/real-estate-simulation/blob/main/ROI_Calculator_Widget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy-financial ipywidgets
from ipywidgets import interact, FloatSlider, IntSlider
import numpy as np
import numpy_financial as npf
import pandas as pd

def 투자_시뮬레이션(매입가, LTV, CapRate, 대출이자율, 대출기간, 보유기간, 법인세율, NOI성장률, ExitCapRate조정):
    대출금 = 매입가 * LTV
    자기자본 = 매입가 - 대출금
    NOI = 매입가 * CapRate
    ExitCapRate = CapRate + ExitCapRate조정

    # 원리금 균등상환
    r = 대출이자율
    n = 대출기간
    연간상환액 = 대출금 * (r * (1 + r)**n) / ((1 + r)**n - 1)
    대출잔액 = 대출금

    현금흐름 = [-자기자본]
    세후이익목록 = []

    for year in range(1, 보유기간+1):
        if year > 1:
            NOI *= (1 + NOI성장률)
        이자 = 대출잔액 * r
        원금 = 연간상환액 - 이자
        대출잔액 -= 원금
        과세소득 = NOI - 이자
        세금 = 과세소득 * 법인세율
        세후이익 = 과세소득 - 세금
        연간현금 = 세후이익 - 원금
        세후이익목록.append(세후이익)
        현금흐름.append(연간현금)

    # Exit 계산
    매각가 = NOI / ExitCapRate
    매각차익 = 매각가 - 대출잔액
    현금흐름[-1] += 매각차익

    # IRR 계산
    IRR = npf.irr(현금흐름)

    # 결과 DataFrame
    df = pd.DataFrame([{
        "자기자본(엔)": int(자기자본),
        "Exit 수익(엔)": int(매각차익),
        "IRR": f"{IRR*100:.2f}%",
        "세후 연평균 배당률": f"{(np.mean(세후이익목록)/자기자본)*100:.2f}%"
    }])
    display(df)

# 위젯 실행
interact(
    투자_시뮬레이션,
    매입가=IntSlider(value=320_000_000, min=50_000_000, max=1_000_000_000, step=10_000_000, description="매입가"),
    LTV=FloatSlider(value=0.6, min=0.3, max=0.9, step=0.05, description="LTV"),
    CapRate=FloatSlider(value=0.03, min=0.02, max=0.06, step=0.005, description="CapRate"),
    대출이자율=FloatSlider(value=0.02, min=0.005, max=0.05, step=0.001, description="이자율"),
    대출기간=IntSlider(value=25, min=5, max=35, step=1, description="대출기간"),
    보유기간=IntSlider(value=10, min=3, max=30, step=1, description="보유기간"),
    법인세율=FloatSlider(value=0.3, min=0.1, max=0.5, step=0.01, description="법인세율"),
    NOI성장률=FloatSlider(value=0.03, min=0.0, max=0.1, step=0.005, description="NOI성장"),
    ExitCapRate조정=FloatSlider(value=-0.005, min=-0.02, max=0.02, step=0.001, description="Exit조정")
)
